# Yashi Game

## Importing relevant libraries

In [ ]:
!pip install python-sat
from pysat.solvers import Minisat22
import itertools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.0 MB/s eta 0:00:00


## Defining useful methods

In [ ]:
def pos_to_clause(n,i,j):
  return n*i+j+1
def edge_to_clause(n,i,j,k,l):
  return (n**2)+1+i*(n**3)+j*(n**2)+k*n+l
def path_to_clause(n,p,path):
  out=n**2+n**4+1
  for i in range(len(path)):
    out+=(path[i]+1)*((p+1)**i)
  return out
def find_all_paths(p):
    return [list(itertools.permutations(range(p), i)) for i in range(2,p+1)]

# Input

In [ ]:
n = 5
#Example 1: n=4 , points=[(0,1),(0,3),(1,0),(1,2),(1,3),(3,1),(3,3)]
#Example 2: n=5, points=[(0,0),(0,2),(0,4),(2,2),(4,0),(4,2),(4,4)]
points = [(0,0),(0,2),(0,4),(2,2),(4,0),(4,2),(4,4)]
p=len(points)
Yashi=Minisat22()

# Clauses implementation

In [ ]:
#Vertices
for i in range(n):
  for j in range(n):
    if (i,j) in points:
      Yashi.add_clause([pos_to_clause(n,i,j)])
    else:
      Yashi.add_clause([-pos_to_clause(n,i,j)])
#Total n^2

#Undirected graph+no self loop+only vertex have edge+only hor/vert segment rule+no intersections:
for i in range(n):
  for j in range(n):
    for k in range(n):
      for l in range(n):
        #Undirected:
        Yashi.add_clause([edge_to_clause(n,i,j,k,l),-edge_to_clause(n,k,l,i,j)]) #+symmetry to do other clause.
        #No self loop:
        if i==k and j==l:
          Yashi.add_clause([-edge_to_clause(n,i,j,k,l)])
        #Only vertex have edge:
        if ((i,j) not in points) or ((k,l) not in points):
          Yashi.add_clause([-edge_to_clause(n,i,j,k,l)])
        #only hor/vert segment rule
        if i!=k and j!=l:
          Yashi.add_clause([-edge_to_clause(n,i,j,k,l)])
        #no intersection
        #Horizontal segment:
        if i==k and j<l:
          #vertical intersection
          for m in range(i):
            for o in range(i+1,n):
              for q in range(j+1,l):
                Yashi.add_clause([-edge_to_clause(n,i,j,k,l), -edge_to_clause(n,m,q,o,q)])
          #horizontal intersection
          for m in range(l):
            for o in range(max(j+1,m+1),n):
              if j!=m or l!=o:
                Yashi.add_clause([-edge_to_clause(n,i,j,k,l), -edge_to_clause(n,i,m,i,o)])
        #Vertical segment:
        if j==l and i<k:
          #horiz intersection
          for m in range(i+1,k):
            for o in range(j):
              for q in range(j+1,n):
                Yashi.add_clause([-edge_to_clause(n,i,j,k,l), -edge_to_clause(n,m,o,m,q)])
          #vert intersection
          for m in range(k):
            for o in range(max(i+1,m+1),n):
              if i!=m or k!=o:
                Yashi.add_clause([-edge_to_clause(n,i,j,k,l), -edge_to_clause(n,m,j,o,j)])

#Total n^2+n^4

#Graph is a tree:

#Auxillary variables:
All_paths=find_all_paths(p)
for i in range(len(All_paths)):
  for j in range(len(All_paths[i])):
    C=[path_to_clause(n,p,All_paths[i][j])]
    for k in range(len(All_paths[i][j])-1):
      C.append(-edge_to_clause(n,points[All_paths[i][j][k]][0],points[All_paths[i][j][k]][1],points[All_paths[i][j][k+1]][0],points[All_paths[i][j][k+1]][1]))
    Yashi.add_clause(C)
    for k in range(len(All_paths[i][j])-1):
        Yashi.add_clause([-path_to_clause(n,p,All_paths[i][j]),edge_to_clause(n,points[All_paths[i][j][k]][0],points[All_paths[i][j][k]][1],points[All_paths[i][j][k+1]][0],points[All_paths[i][j][k+1]][1])])

#Exists a path from i to j
for i in range(len(points)):
  for j in range(len(points)):
    if i!=j:
      C=[]
      for k in range(len(All_paths)):
        for l in range(len(All_paths[k])):
          if i==All_paths[k][l][0] and j==All_paths[k][l][len(All_paths[k][l])-1]:
            C.append(path_to_clause(n,p,All_paths[k][l]))
      Yashi.add_clause(C)

#No cycles
for i in range(len(points)):
  C=[]
  for j in range(len(All_paths)):
    for k in range(len(All_paths[j])):
      if i==All_paths[j][k][0] and len(All_paths[j][k])>2:
        Yashi.add_clause([-path_to_clause(n,p,All_paths[j][k]),-edge_to_clause(n,points[All_paths[j][k][len(All_paths[j][k])-1]][0],points[All_paths[j][k][len(All_paths[j][k])-1]][1],points[i][0],points[i][1])])


#Solution

In [ ]:
#Print 1 solution
print(Yashi.solve())
for i in Yashi.get_model():
  if i>n**2 and i<=n**2+n**4:
    z=i-n**2-1
    #print(z)
    l=z%n
    z=int((z-l)/n)
    k=z%n
    z=int((z-k)/n)
    j=z%n
    z=int((z-j)/n)
    i=z
    print(i,j,k,l)

True
0 1 0 3
0 3 0 1
0 3 1 3
1 0 1 2
1 2 1 0
1 2 1 3
1 3 0 3
1 3 1 2
1 3 3 3
3 1 3 3
3 3 1 3
3 3 3 1


In [ ]:
#Print best solution
best_total=-1
while Yashi.solve():
  C=[]
  Y=Yashi.get_model()
  total=0
  for i in Y:
    if i>n**2 and i<=n**2+n**4:
      C.append(-i)
      z=i-n**2-1
      l=z%n
      z=int((z-l)/n)
      k=z%n
      z=int((z-k)/n)
      j=z%n
      z=int((z-j)/n)
      ii=int(z)
      if ii==k:
        total=total+max(j,l)-min(j,l)
      else:
        total=total+max(ii,k)-min(ii,k)
    if i>n**2+n**4:
      break
  if best_total==-1 or total<best_total:
    best_total=total
    best_model=Y
  Yashi.add_clause(C)
for i in best_model:
  if i>n**2 and i<=n**2+n**4:
    z=i-n**2-1
    l=z%n
    z=int((z-l)/n)
    k=z%n
    z=int((z-k)/n)
    j=z%n
    z=int((z-j)/n)
    i=z
    print(i,j,k,l)

0 1 0 3
0 3 0 1
0 3 1 3
1 0 1 2
1 2 1 0
1 2 1 3
1 3 0 3
1 3 1 2
1 3 3 3
3 1 3 3
3 3 1 3
3 3 3 1


In [ ]:
print(best_total/2)

10.0
